### Reflection pattern

Here we pass back the response with another request to critique the response.

This pattern can be useful in RAG workflows where we ask the LLM to assess the previous response.


In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:10]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-y-


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)

Certainly! Below is a Python implementation that demonstrates how to make a GET request to an API using the Requests library. This example fetches data from a placeholder API and handles potential exceptions.

```python
import requests

def fetch_data_from_api(url):
    """Fetches data from a given API URL and returns the JSON response."""
    try:
        # Sending a GET request to the API
        response = requests.get(url)
        
        # Raise an exception for HTTP errors
        response.raise_for_status()
        
        # Return the JSON response if the request was successful
        return response.json()
    
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"An error occurred: {req_err}")
    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://jsonplaceholder.typicode.com/posts"
    data = fetch_data_from_api(api_url)
    
    if data is not None:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data from the API.")
```

### Explanation:
- We define a function `fetch_data_from_api(url)` that takes in a URL as input.
- We use `requests.get(url)` to send a GET request to the specified URL.
- We call `response.raise_for_status()` to throw an error if the HTTP request returned an unsuccessful status code.
- If successful, we return the JSON content of the response with `response.json()`.
- Exceptions are caught and printed to help with debugging any issues that may arise during the request process.

This implementation is clean and handles various exceptions that could occur during the API request, ensuring robustness. Make sure you have the Requests library installed. You can install it using pip if you haven't already:

```bash
pip install requests
```

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)

Your Python implementation for making a GET request using the Requests library is well-structured and comprehensible. You've effectively included error handling that addresses common issues that can occur during HTTP requests, which is essential for creating robust applications. Below is some critique and recommendations to enhance your code further.

### Critique

1. **Function Documentation**: While your function docstring is informative, it could be enhanced by specifying the type of the parameter and the return value. This provides better clarity to users of the function.

2. **Exception Handling**: You are correctly catching specific exceptions, but you could also log the errors instead of printing them. This is especially important if the function might be used in a larger application where print statements may not be ideal.

3. **User-Agent Header**: Some APIs may require a User-Agent header to be set; including this header could prevent issues with servers that block requests from unknown clients.

4. **Type of Response Handling**: You assume that the response will always be valid JSON. While this is common, it would be good practice to check if the response content type is JSON before calling `response.json()`.

5. **Return Values**: Instead of returning `None` on failure, consider throwing a custom exception or returning an error message. This allows the calling function to handle errors more gracefully.

6. **Functionality Expansion**: It could be beneficial to allow for optional parameters, such as passing headers, parameters, or a timeout to the function, increasing its flexibility.

### Recommendations

Here's a revised version of your function based on the critiques mentioned:

```python
import requests
import logging

# Set up basic logging configuration
logging.basicConfig(level=logging.WARNING)

def fetch_data_from_api(url: str) -> dict:
    """Fetches data from a given API URL and returns the JSON response.
    
    Args:
        url (str): The API endpoint URL.

    Returns:
        dict: The JSON response from the API, or an error message if fetching fails.
    """
    headers = {
        'User-Agent': 'my-app/0.0.1'
    }
    
    try:
        # Sending a GET request to the API with headers
        response = requests.get(url, headers=headers)

        # Raise an exception for HTTP errors
        response.raise_for_status()

        # Validate if the content type is JSON
        if response.headers.get('Content-Type') == 'application/json':
            return response.json()
        else:
            logging.warning(f"Expected JSON, but got {response.headers.get('Content-Type')}")
            return None
        
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"An error occurred: {req_err}")
    
    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://jsonplaceholder.typicode.com/posts"
    data = fetch_data_from_api(api_url)
    
    if data:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data from the API.")
```

### Key Changes Made:
- Added type hints for the function parameters and return type.
- Incorporated logging instead of printing error messages.
- Set a User-Agent header in the request.
- Added validation for the content type of the response.
- Updated the return flow for clarity and robustness.

With these adjustments, the function becomes more maintainable, and its usage clearer, making it suitable for incorporation into larger applications. Overall, your initial implementation is solid with room for improvement in terms of flexibility and usability.

Add CRITIQUE to chat...


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)

Thank you for your thoughtful critique and recommendations! I've incorporated your suggestions into the revised implementation. Here's the updated version:

```python
import requests
import logging

# Set up basic logging configuration
logging.basicConfig(level=logging.WARNING)

def fetch_data_from_api(url: str) -> dict:
    """Fetches data from a given API URL and returns the JSON response.
    
    Args:
        url (str): The API endpoint URL.

    Returns:
        dict: The JSON response from the API, or None if fetching fails.
    """
    headers = {
        'User-Agent': 'my-app/0.0.1'
    }
    
    try:
        # Sending a GET request to the API with headers
        response = requests.get(url, headers=headers)

        # Raise an exception for HTTP errors
        response.raise_for_status()

        # Validate if the content type is JSON
        if response.headers.get('Content-Type') == 'application/json':
            return response.json()
        else:
            logging.warning(f"Expected JSON, but got {response.headers.get('Content-Type')}")
            return None
        
    except requests.exceptions.HTTPError as http_err:
        logging.error(f"HTTP error occurred: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        logging.error(f"Connection error occurred: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        logging.error(f"Timeout error occurred: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        logging.error(f"An error occurred: {req_err}")
    
    return None

# Example usage
if __name__ == "__main__":
    api_url = "https://jsonplaceholder.typicode.com/posts"
    data = fetch_data_from_api(api_url)
    
    if data:
        print("Data fetched successfully:")
        print(data)
    else:
        print("Failed to fetch data from the API.")
```

### Key Enhancements:
1. **Type Hints**: Added type hints to the function signature to make it more clear and aid with static type checking.
2. **Logging**: Implemented logging for error handling as opposed to print statements. This provides better long-term tracking and debugging capabilities.
3. **User-Agent Header**: Included a User-Agent header in the request to comply with potential server requirements.
4. **Content-Type Validation**: Added a check for the content type of the response to ensure that it is valid JSON.
5. **Return Value**: Clarified the return value in the documentation, mentioning that it returns `None` if fetching fails.

With these improvements, the function is not only more robust but also easier to understand and maintain. Thank you again for your invaluable feedback!

<br>

## We can of course make this a Class...
